In [84]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.dummies import dummify_columns

In [86]:
data = pd.concat([
    pd.read_csv("../processed_data/bars.csv", index_col=0),
    pd.read_csv("../processed_data/myrealty.csv", index_col=0),
    pd.read_csv("../processed_data/bnakaran.csv", index_col=0)
], axis = 0)
data = data.reset_index(drop=True)

In [2]:
scraping_log = pd.read_csv("../scraping_results/scraping_log.csv", index_col=0)
rents = scraping_log[scraping_log.webpage.str.contains("rent")]
rents = rents[rents.success == True]
bnakaran = pd.read_csv("../scraping_results/data/bnakaran_apartments.csv", index_col=0)
myrealty = pd.read_csv("../scraping_results/data/myrealty_apartments.csv", index_col=0)

renting_ap = bnakaran[bnakaran.webpage.isin(rents.webpage)]
filtered_data = data[(data["source"] == 'bnakaran') & (data["id"].isin(renting_ap["id"]))]
data = data.drop(filtered_data.index)

renting_ap = myrealty[myrealty.webpage.isin(rents.webpage)]
renting_ap.id = renting_ap.id.astype(str)
filtered_data = data[(data["source"] == 'myrealty') & (data["id"].isin(renting_ap["id"]))]
data = data.drop(filtered_data.index)

NameError: name 'pd' is not defined

In [87]:
prices_full = data["price"] > 30000
data.loc[prices_full, "price"] = data.loc[prices_full, "price"] / data.loc[prices_full, "area"]

In [88]:
data = pd.get_dummies(data, columns = ["building_type", "condition"])

In [90]:
data.to_csv("../processed_data/data_no_map.csv")